In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sea 

In [2]:
import re 
import nltk

In [3]:
from nltk.tokenize import word_tokenize,sent_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer

In [4]:
stemmer=PorterStemmer()
lemitize=WordNetLemmatizer()

In [5]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [27]:
df=pd.read_csv('train.csv')

In [28]:
df.shape

(20800, 5)

In [29]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [31]:
df['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [32]:
df=df[['title','label']]

In [33]:
df.head()

,title,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [34]:
# searching for all the empty value in the dataframe 

df.isnull().sum()

title    558
label      0
dtype: int64

In [35]:
# droping the null value 

df.dropna(inplace=True)

In [36]:
# making all the value of title in lowercase

df['title']=df['title'].apply(lambda x:x.lower())

In [37]:
# apply all data cleaning techinique

def cleaning(text):
    var1=[]
    for word in word_tokenize(text):
        var1.append(stemmer.stem(word))
    var2=[]
    for word in var1:
        var2.append(lemitize.lemmatize(word))
    return var2    

In [38]:
df['title']=df['title'].apply(lambda x:cleaning(x))

In [44]:
df.head()

,title,label
0,hous dem aid : we didn ’ t even see comey ’ s ...,1
1,"flynn : hillari clinton , big woman on campu -...",0
2,whi the truth might get you fire,1
3,15 civilian kill in singl u airstrik have been...,1
4,iranian woman jail for fiction unpublish stori...,1


In [40]:
#joining the word of title 1

df['title']=df['title'].apply(lambda x:' '.join(x))

In [41]:
# clasifing the dataset in form of input(x) and output(y)

x=df['title']
y=df['label']

In [45]:
# import libraies for bag of words and tfidf

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

bag=CountVectorizer(stop_words='english')
tfidf=TfidfVectorizer(stop_words='english',ngram_range=(2,3))

In [46]:
# transformation in form of bag of words

x_bag=bag.fit_transform(x)

In [47]:
# tranformation in form of tfidf

x_tfidf=tfidf.fit_transform(x)

In [48]:
# imposing train test split for bag of word

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_bag,y,random_state=0,test_size=0.25)

In [50]:
# appling linearclassification model

from sklearn.linear_model import LogisticRegression

model=LogisticRegression()
model.fit(x_train,y_train)

y_pred=model.predict(x_test)

In [52]:
# checking the accuracy of the model

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9369689784627544

In [54]:
# imposing train test split for tfidf 

x_train,x_test,y_train,y_test=train_test_split(x_tfidf,y,random_state=0,test_size=0.25)

In [55]:
# appling linearclassification model

model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [56]:
#checking the accuracy of the model

accuracy_score(y_test,y_pred)

0.7968780873345189

# using neural network and LSTM

In [57]:
df.head()

,title,label
0,hous dem aid : we didn ’ t even see comey ’ s ...,1
1,"flynn : hillari clinton , big woman on campu -...",0
2,whi the truth might get you fire,1
3,15 civilian kill in singl u airstrik have been...,1
4,iranian woman jail for fiction unpublish stori...,1


In [59]:
#stop word removal 

stop_word=stopwords.words('english')
def stop(text):
    var1=[]
    for i in word_tokenize(text):
        if i not in stop_word:
            var1.append(i)
    return ' '.join(var1)

In [60]:
df['title']=df['title'].apply(lambda x:stop(x))

In [68]:
# applying re

def reg(text):
    text=re.sub('[^a-zA-Z]',' ',text)
    return text

In [69]:
df['title']=df['title'].apply(lambda x:reg(x))

In [70]:
x=df['title']
y=df['label']

In [71]:
import tensorflow as tf

In [72]:
from tensorflow.keras.layers import Embedding,Dense,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [73]:
voc_size=10000

In [75]:
corpus=[]
for i in x:
    corpus.append(i)

In [77]:
# one hot representaion of data 

onehot_rep=[one_hot(word,voc_size) for word in corpus]

In [78]:
onehot_rep

[[2914, 170, 1497, 3743, 2136, 1446, 6994, 9685, 6354, 1182],
 [1437, 7903, 8337, 7120, 172, 9917, 9833],
 [8548, 7114, 2114, 6818, 9242],
 [5579, 8507, 2328, 652, 5302, 8288],
 [7981, 172, 8037, 690, 7251, 4820, 172, 6992, 1255, 2560],
 [3987,
  7127,
  9597,
  9844,
  8244,
  9414,
  4400,
  2054,
  5982,
  8011,
  7237,
  9019,
  7244,
  5656,
  9833],
 [4361, 4361, 9294, 7730, 1043, 4327, 7629, 45, 9935, 3266, 1113, 9359, 2663],
 [9134, 6815, 9976, 3006, 3395, 3479, 2721, 5916, 5355, 5721, 8574, 9334],
 [693, 8875, 9137, 8376, 9414, 1042, 3107, 4470, 5722, 8399, 5721, 8574, 9334],
 [238, 7350, 621, 9583, 3418, 1363, 9414, 5021, 5721, 8574, 9334],
 [4059, 9037, 3321, 1336, 1900, 6329, 3221, 1704, 9414, 3459],
 [1692, 9060, 285, 7911, 7588, 3400, 4422, 3113],
 [8867, 6122, 1775, 3072, 2113, 2724, 7024, 1622, 4244, 3688, 5025],
 [652, 5832, 2136, 6329, 9414, 3418],
 [1478, 1768, 5913, 1218, 5307, 9539, 765, 3135, 1910],
 [9712, 5128, 6427, 1853, 7882, 7957, 1637, 5721, 8574, 9334],
 [

In [79]:
# embedding representaion

sent_length=20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)

In [80]:
print(embedded_docs)

[[   0    0    0 ... 9685 6354 1182]
 [   0    0    0 ...  172 9917 9833]
 [   0    0    0 ... 2114 6818 9242]
 ...
 [   0    0    0 ... 5721 8574 9334]
 [   0    0    0 ...  214 6004 9502]
 [   0    0    0 ... 6512 9744  917]]


In [81]:
len(embedded_docs)

20242

In [82]:
embedded_docs.shape

(20242, 20)

In [84]:
# modeling
 
feature =50
model=Sequential()
model.add(Embedding(voc_size,feature,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [85]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 560,501
Trainable params: 560,501
Non-trainable params: 0
_________________________________________________________________


In [86]:
x_array=np.array(embedded_docs)
y=df['label']

In [88]:
x_train,x_test,y_train,y_test=train_test_split(x_array,y,random_state=42,test_size=0.33)

In [89]:
# model training

model.fit(x_train,y_train,epochs=20,batch_size=64)

Epoch 1/20
212/212 [==============================] - 29s 54ms/step - loss: 0.3213 - accuracy: 0.8576
Epoch 2/20
212/212 [==============================] - 13s 59ms/step - loss: 0.1192 - accuracy: 0.9541
Epoch 3/20
212/212 [==============================] - 13s 62ms/step - loss: 0.0712 - accuracy: 0.9764
Epoch 4/20
212/212 [==============================] - 11s 54ms/step - loss: 0.0376 - accuracy: 0.9878
Epoch 5/20
212/212 [==============================] - 13s 59ms/step - loss: 0.0201 - accuracy: 0.9939
Epoch 6/20
212/212 [==============================] - 14s 67ms/step - loss: 0.0104 - accuracy: 0.9975
Epoch 7/20
212/212 [==============================] - 16s 74ms/step - loss: 0.0041 - accuracy: 0.9990
Epoch 8/20
212/212 [==============================] - 14s 65ms/step - loss: 0.0019 - accuracy: 0.9996
Epoch 9/20
212/212 [==============================] - 14s 67ms/step - loss: 0.0035 - accuracy: 0.9987
Epoch 10/20
212/212 [==============================] - 16s 76ms/step - loss: 0.001

  ## it seems that it is a case of overfitting so we can implement the above model also with the inclusion of dropout